# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
!pip install opendatasets

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import Dataset
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil
import opendatasets as od
import pandas as pd
from azureml.widgets import RunDetails

In [10]:
ws = Workspace.from_config()
experiment_name = 'udacity_capstone_hyperdrive2'

experiment = Experiment(ws, experiment_name)

In [4]:
cluster_name = "IntensePurposeCluster"

try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    print('existing cluster found')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_cluster.wait_for_completion(show_output=True)

existing cluster found


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
#Check if data exists, else download
if not os.path.exists('covid19-dataset/Covid Data.csv'):
    # Download kaggle covid dataset
    print('downlaoding data')
    od.download('https://www.kaggle.com/datasets/meirnizri/covid19-dataset/', '.')

In [6]:
df = pd.read_csv(r'./covid19-dataset/Covid Data.csv')

In [7]:
#Transform data to get correct format
df['y'] = (df['DATE_DIED'] != '9999-99-99').astype(int)
df['SEX'] = df['SEX'] - 1
df['hospitalized'] = df['PATIENT_TYPE'] - 1
df['tested_positive'] = (df['CLASIFFICATION_FINAL']<4).astype(int)
df = df.drop(columns=['DATE_DIED', 'PATIENT_TYPE', 'CLASIFFICATION_FINAL'])

cols = ['INTUBED', 'PNEUMONIA', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'ICU']

for col in cols:
    df[col] = (df[col] == 1).astype(int)

In [8]:
df.head(9)

,USMER,MEDICAL_UNIT,SEX,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,...,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,ICU,y,hospitalized,tested_positive
0,2,1,0,0,1,65,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
1,2,1,1,0,1,72,0,0,0,0,...,1,0,0,1,1,0,0,1,0,0
2,2,1,1,1,0,55,0,1,0,0,...,0,0,0,0,0,0,0,1,1,1
3,2,1,0,0,0,53,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2,1,1,0,0,68,0,1,0,0,...,1,0,0,0,0,0,0,1,0,1
5,2,1,0,0,1,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6,2,1,0,0,0,64,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,2,1,0,0,1,64,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
8,2,1,0,0,0,37,0,1,0,0,...,1,0,0,1,0,0,0,0,1,1


In [9]:
datastore = ws.get_default_datastore()

dataset = Dataset.Tabular.register_pandas_dataframe(df,
                target=datastore, 
                name='capstone_dataset', 
                description='Input dataset (Kaggle covid data) for capstone project')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/ef45cbe1-b5a4-4a69-b0d2-83991dd3069d/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
# Utilising a simple Bandit Policy for early termination in case of stagnant performance
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

# Defining a search space for the optimal parameters (GradientBoostingTree)
param_sampling = RandomParameterSampling({'learning_rate':uniform(0.01, 1.0),
                             'n_estimators': choice(50, 100, 150, 200, 250, 300, 500),
                             "subsample": choice(0.5, 0.9, 1.0)})

# Create directories to ensure correct model tracking
if "training" not in os.listdir():
    os.mkdir("./training")

if experiment_name not in os.listdir():
    os.mkdir("./" + experiment_name + '/')
    shutil.copy2('./covdata_train.py', './'+ experiment_name + '/')

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# Using script run instead of estimator for the hyperdrive run
src = ScriptRunConfig(source_directory='./'+ experiment_name + '/', 
                      script='covdata_train.py', 
                      compute_target=compute_cluster,
                     arguments=['--learning_rate',0.1, '--n_estimators', 100, '--subsample', '1.0'],
                     environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name='F1Score',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    max_concurrent_runs=4)

In [12]:
# Submiting the experiment
hdr = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hdr).show()
hdr.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_0b815141-118c-46ce-95fb-b836a2cd9b3c',
 'target': 'IntensePurposeCluster',
 'status': 'Completed',
 'startTimeUtc': '2023-01-25T09:16:30.412183Z',
 'endTimeUtc': '2023-01-25T09:57:53.691153Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"F1Score","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '61aacf74-dc5a-4aa2-85e9-9bbc1624b675',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.6019377914610403',
  'best_child_run_id': 'HD_0b815141-118c-46ce-95fb-b836a2cd9b3c_41',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_0b815141-118c-46ce-95fb-b836a2cd9b3c_41'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'te

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
# Get the best model according to metric and register as model
best_run = hdr.get_best_run_by_primary_metric()
best_run.register_model('covid_death_pred_hdr', 'outputs/Hyperdrive_capstone.joblib')

#Complete run
hdr.complete()

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [50]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'covid_death_pred_hdr')

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
inference_config = InferenceConfig(entry_script="covdata_entry.py", environment=sklearn_env)

# Deploy locally first
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(workspace=ws, 
                       name='covidsurvival', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config = deployment_config)

local_service.wait_for_deployment(show_output=True)
print(f"Scoring URI is : {local_service.scoring_uri}")

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io
Logging into Docker registry 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 3fc95e45fb7841ebb16dd39c91442edf.azurecr.io/azureml/azureml_4fada07952d2b43d666b6b63633ffdd5
 ---> 1e1b829da9ea
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 979c19e6e1f1
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjFiOTQ0YTliLWZkYWUtNGY5Ny1hZWIxLWI3ZWVhMGJlYWM1MyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTIyMzQ2NCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTIyMzQ2NCIsIndvcmtzcGFjZUlkIjoiM2ZjOTVlNDUtZmI3OC00MWViLWIxNmQtZDM5YzkxNDQyZWRmIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in e7c5d9f0b11f
 ---> f8af836ad45a
Step 4/5 : RUN mv '/var/azureml-app/tmp_wofhmjy.py' /var/azureml-app/main.py


TODO: In the cell below, send a request to the web service you deployed to test it.

In [48]:
import json

data = {"data": [
          {
            "USMER": 2,
            "MEDICAL_UNIT": 1, 
            "SEX": 1,
            "INTUBED": 0,
            "PNEUMONIA": 0,
            "AGE": 35,
            "PREGNANT": 0,
            "DIABETES": 1,
            "COPD": 0,
            "ASTHMA": 0,
            "INMSUPR": 0, 
            "HIPERTENSION": 0,
            "OTHER_DISEASE": 0,
            "CARDIOVASCULAR": 0,
            "OBESITY": 1,
            "RENAL_CHRONIC": 0,
            "TOBACCO": 0,
            "ICU": 0, 
            "hospitalized": 0,
            "tested_positive": 0
          }]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

In [52]:
import requests

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://localhost:6789/score'
# If the service is authenticated, set the key or token
#key = ''

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


[0]


TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

